In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D
import numpy as np
import pandas as pd
import keras
from keras.callbacks import EarlyStopping
from keras import backend as K
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,mean_absolute_percentage_error

In [ ]:
import os

def get_company_names(directory):
  company_names = []
  for filename in os.listdir(directory):
    if filename.endswith("_model.h5"):
      company_name = filename[:-9]  # Remove "_model.h5"
      company_names.append(company_name)
  return company_names

# Example usage (assuming your directory is correct):
directory_path = "/content/drive/MyDrive/models"
company_names_list = get_company_names(directory_path)
# print(company_names_list)
print(len(company_names_list))

32


In [ ]:
all_prediction = {}
eval = {}
for company in company_names_list:
    K.clear_session()
    df = pd.read_csv(f'/content/drive/MyDrive/stocks_new/{company}.csv')

    prices = df['Close'].values.tolist()

    def clean_stock_prices(stock_prices):
        cleaned_prices = []
        for price in stock_prices:

            if isinstance(price, str):
                price = price.replace(',', '').strip()  # Remove commas
            try:
                price = float(price)
                if not np.isnan(price):  # Check for NaN values
                    cleaned_prices.append(price)
            except ValueError:
                pass  # Skip invalid values

        return np.array(cleaned_prices).reshape(-1,1)

    prices = clean_stock_prices(prices)

    scaler = MinMaxScaler()
    scaled_prices = scaler.fit_transform(prices)

    def create_sequences(data, sequence_length):
        sequences = []
        labels = []
        for i in range(len(data) - (sequence_length+30)):
            sequences.append(data[i:i+sequence_length])
            labels.append(data[i+sequence_length+30])

        return np.array(sequences), np.array(labels)

    sequence_length = 60
    X, y = create_sequences(scaled_prices, sequence_length)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Building the LSTM model
    model = load_model(f'/content/drive/MyDrive/models/{company}_model.h5')

    p_test = model.predict(X)
    p_test = scaler.inverse_transform(p_test)
    actual_prices = scaler.inverse_transform(y.reshape(-1,1))

    r2 = r2_score(actual_prices, p_test)
    mse = mean_squared_error(actual_prices, p_test)
    mae = mean_absolute_error(actual_prices, p_test)
    mape = mean_absolute_percentage_error(actual_prices, p_test)
    variance = np.var(actual_prices)

    all_prediction[company] = p_test.flatten().tolist()

    eval[company] = {
        "r2": r2,
        "mse": mse,
        "mae": mae,
        "mape": mape,
        "variance": variance
    }



163/163 [==============================] - 14s 80ms/step


In [ ]:
import json
with open('eval.json', 'w') as file:
    json.dump(eval, file)
with open('prediction.json', 'w') as file:
    json.dump(all_prediction, file)

In [ ]:
# prompt: download eval.json and prediction.json

from google.colab import files
files.download('eval.json')
files.download('prediction.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

K.clear_session()
df = pd.read_csv(f'/content/drive/MyDrive/stocks_new/ADANIPORTS.csv')

prices = df['Close'].values.tolist()

def clean_stock_prices(stock_prices):
    cleaned_prices = []
    for price in stock_prices:

        if isinstance(price, str):
            price = price.replace(',', '').strip()  # Remove commas
        try:
            price = float(price)
            if not np.isnan(price):  # Check for NaN values
                cleaned_prices.append(price)
        except ValueError:
            pass  # Skip invalid values

    return np.array(cleaned_prices).reshape(-1,1)

prices = clean_stock_prices(prices)

scaler = MinMaxScaler()
scaled_prices = scaler.fit_transform(prices)

def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - (sequence_length+30)):
        sequences.append(data[i:i+sequence_length])
        labels.append(data[i+sequence_length+30])

    return np.array(sequences), np.array(labels)

sequence_length = 60
X, y = create_sequences(scaled_prices, sequence_length)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Building the LSTM model
model = load_model(f'/content/drive/MyDrive/models/ADANIPORTS_model.h5')

p_test = model.predict(X)
p_test = scaler.inverse_transform(p_test)
actual_prices = scaler.inverse_transform(y.reshape(-1,1))
r2 = r2_score(actual_prices, p_test)
mse = mean_squared_error(actual_prices, p_test)
mae = mean_absolute_error(actual_prices, p_test)
mape = mean_absolute_percentage_error(actual_prices, p_test)
variance = np.var(actual_prices)
# all_prediction[company] = p_test

# print("Variance of actual prices:", variance)
# print(f'R-squared (R²): {r2}')
# print(f'Mean Squared Error (MSE): {mse}')
# print(f'Mean Absolute Error (MAE): {mae}')
# print(f'Mean Absolute Percentage Error (MAPE): {mape}')
print(p_test)

129/129 [==============================] - 9s 56ms/step
[[ 127.90678]
 [ 129.05087]
 [ 129.44586]
 ...
 [1273.1616 ]
 [1273.9932 ]
 [1272.8856 ]]


In [ ]:
p_test.flatten().tolist()

[127.90677642822266,
 129.05087280273438,
 129.44586181640625,
 129.2410888671875,
 130.8660125732422,
 133.54161071777344,
 137.60523986816406,
 140.57965087890625,
 140.35987854003906,
 139.31446838378906,
 136.81283569335938,
 134.9005889892578,
 133.73287963867188,
 135.92759704589844,
 137.39344787597656,
 137.11720275878906,
 136.0390625,
 133.14285278320312,
 130.65826416015625,
 130.02883911132812,
 127.54997253417969,
 128.8008575439453,
 129.13021850585938,
 131.3563232421875,
 132.9401092529297,
 141.00668334960938,
 142.346923828125,
 145.0535125732422,
 145.43382263183594,
 144.44500732421875,
 143.21119689941406,
 140.9127960205078,
 139.91893005371094,
 138.24203491210938,
 138.44862365722656,
 137.31463623046875,
 139.59725952148438,
 139.224365234375,
 139.27418518066406,
 139.06837463378906,
 140.3939208984375,
 141.1224365234375,
 141.1315155029297,
 144.00308227539062,
 147.00381469726562,
 149.1273956298828,
 150.98385620117188,
 151.56500244140625,
 150.6510009765